In [1]:
import os
for env in ["MKL_NUM_THREADS", "NUMEXPR_NUM_THREADS", "OMP_NUM_THREADS"]:
    os.environ[env] = "1"

import numpy as np
from numpy.random import default_rng
import numpy.random as rnd
from ttictoc import tic, toc

import approxbayescomp as abc

In [2]:
import sys

print("ABC version:", abc.__version__)
print("Python version:", sys.version)
print("Numpy version:", np.__version__)

tic()

ABC version: 0.1.0
Python version: 3.8.10 | packaged by conda-forge | (default, May 11 2021, 06:25:29) 
[GCC 9.3.0]
Numpy version: 1.21.0


In [3]:
numIters = 10
popSize = 1000
smcArgs = {"verbose": True, "numProcs": 64}

# Time dependent: nonhomogenous Poisson process with seasonality

The claim arrival process is governed by a non homogenous Poisson process $N_t$ with intensity function 

$$
\lambda(t) = b[\sin(2\pi c t) + 1]
$$

The claim frequency data $n_s,\,s = 1,\ldots, t$ correspond to the increments of this non homogeneous Poisson process. These are independent Poisson variables $\text{Pois}[\mu(s)]$ with 

$$
\mu(s) = \int_{s-1}^s\lambda(u)du = b + \frac{b}{2\pi c}\left[\cos(2\pi(s-1) c) - \cos(2\pi s c)\right],\quad s = 1,\ldots, t
$$



In [4]:
rg = default_rng(123)
sample_sizes = [50, 250]
T = sample_sizes[-1]

a, b, c, μ, σ = 1, 5, 1 / 50, 0, 0.5

θ_True = a, b, c, μ, σ

freq, sev, theta = "cyclical_poisson", "lognormal", [a, b, c, μ, σ]

freqs, sevs = abc.simulate_claim_data(rg, T, freq, sev, theta)

# Aggregation process
psi = abc.Psi("sum")

xData = abc.compute_psi(freqs, sevs, psi)

In [5]:
params = ("a", "b", "c", "\mu", "\sigma")
prior = abc.IndependentUniformPrior(
    [(0, 50), (0, 50), (1 / 1000, 1 / 10), (-10, 10), (0, 3)]
)
model = abc.Model("cyclical_poisson", "lognormal", psi, prior)

In [6]:
%%time 
AR = 2

for ss in sample_sizes:
    xDataSS = xData[:ss]

    gamma = (np.max(xDataSS) - np.min(xDataSS)) / (ss - 1) * AR

    %time fit2 = abc.smc(numIters, popSize, xDataSS, model, sumstats=abc.wrap_ss_curve_matching(gamma),\
                          distance=abc.wass_2Ddist, **smcArgs)

Final population dists <= 1.77, ESS = 805
CPU times: user 19.8 s, sys: 1.05 s, total: 20.8 s
Wall time: 28.8 s


Final population dists <= 1.73, ESS = 593
CPU times: user 18.5 s, sys: 788 ms, total: 19.3 s
Wall time: 1min 7s
CPU times: user 38.3 s, sys: 1.83 s, total: 40.2 s
Wall time: 1min 36s


In [7]:
elapsed = toc()
print(f"Notebook time = {elapsed:.0f} secs = {elapsed/60:.2f} mins")

Notebook time = 97 secs = 1.62 mins
